## 2g. Evidence - Inclusivity QAS Measurements

Evidence collected in this section checks for the inclusivity QAS scenario defined in the previous step. Note that some functions and data will be loaded from external Python files.

The cell below must contain JSON data about this evidence that will be used to automatically populate the sample test catalog.

In [ ]:
{
    "tags": ["LLM", "content generation"],
    "quality_attribute": "inclusivity",
    "description": "an assessment on if the generated reviews are influenced by employee writing levels",
    "inputs": "A request for a review in a crafted prompt, with the supporting material (employee statement, goals and objectives, and manager comments), where the writing levels of the emplpoyee statements have been edited to be of a wide range",
    "output": "ANOVA results on extracted overall evaluation scores from the generated reviews",
}

### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces. This import will also set up global constants related to folders and model to use.

In [ ]:
# Sets up context for the model being used, sets up constants related to folders and model data to be used.
from session import *

### Set up scenario test case

In [ ]:
from mlte.negotiation.artifact import NegotiationCard

card = NegotiationCard.load()
qa = 7
print(card.quality_scenarios[qa])

### A Specific test case generated from the scenario:

**Data and Data Source:**	The self evaluations in the original test data set will be used to generate a number of contextually similar but reading level different self-evaluations, which all convey the same information but have dramatically different Flesch-Kincade grade level or Flesch reading ease score. 

**Measurement and Condition:**	The evaluation text should be contextually similar across each evaluation set, but the readability of the text should change. The influence of the readability will be measured using 2-way ANOVA (with prompt group and a readability score being the 2 factors), with significance of p<0.05. The text readability will be measured by the Flesch-Kincade Grade level or the Flesch Reading Ease score.

**Context:**	Normal Operation

### Gather evidence

In [ ]:
import numpy as np
import pandas as pd
from os import path

import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
# Read the files with with the necessary input data and LLM evaluation results
input_df = pd.read_csv(
    os.path.join(DATASETS_DIR, "2h_llm_input_inclusivity.csv")
)

output_df = pd.read_csv(
    path.join(DATASETS_DIR, "2h_llm_output_inclusivity.csv")
)  # data file with LLm results

combo_df = pd.merge(
    input_df, output_df, left_on="Unnamed: 0", right_on="Unnamed: 0"
)

combo_df = combo_df[
    [
        "evaluationOutput",
        "extractedOverallRating",
        "PromptGroupNum",
        "Flesch-Kincade Grade Level",
        "Flesch Reading Ease Score",
    ]
]
combo_df.rename(
    columns={
        "extractedOverallRating": "overallRating",
        "Flesch-Kincade Grade Level": "FKGrade",
        "Flesch Reading Ease Score": "FReadingScore",
    },
    inplace=True,
)

combo_df.head()

In [ ]:
# take a subset of teh data; and make sure are the right type
combo_df2 = combo_df[
    ["overallRating", "PromptGroupNum", "FKGrade", "FReadingScore"]
]
combo_df2 = combo_df2.astype(
    {
        "overallRating": int,
        "PromptGroupNum": str,
        "FKGrade": float,
        "FReadingScore": float,
    }
)
combo_df2

### Save evidence to the specicified scenario

In [ ]:
# run test, collect p-values


def run_statsmodel_lm(combo_df2):

    model = ols(
        "overallRating ~ C(PromptGroupNum) + FReadingScore+ C(PromptGroupNum):FReadingScore",
        data=combo_df2,
    ).fit()
    res = sm.stats.anova_lm(model, typ=2)

    print(res)
    if res["PR(>F)"].loc["FReadingScore"] < 0.05:
        print("fail test")
    else:
        print("pass test")

    f_rs = res["F"].loc["FReadingScore"]
    p_rs = res["PR(>F)"].loc["FReadingScore"]

    return [float(p_rs)]


res = run_statsmodel_lm(combo_df2)
print(res)

In [ ]:
from mlte.evidence.types.array import Array
from mlte.measurement.external_measurement import ExternalMeasurement

am_measurement = ExternalMeasurement(
    "eval not dependent on writing level", Array, run_statsmodel_lm
)

# evaluate
result = am_measurement.evaluate(combo_df2)

print(result)
result.save(force=True)